In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
model= ChatOpenAI()

In [12]:
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluate: str

In [4]:
def outline_creation(state:BlogState) -> BlogState:
    title = state['title']

    prompt= f"""Give me a detailed about {title}. Do Not provide wrong information validate and give the source of information. If Not able to get" \
    correct information please say don't know"""

    outline = model.invoke(prompt).content
    print("111111111111",outline)
    state['outline']= outline

    return state

In [5]:
def content_creation(state:BlogState) -> BlogState:
    title= state['title']
    outline = state['outline']

    prompt= f"""Write a blog on {title} using below information:\n {outline}. If you found better source than i provided to you please use it and keep that"
    "source in the blog"""

    content = model.invoke(prompt).content

    state['content']= content
    
    return state

In [13]:
def evaluate_response(state: BlogState) -> BlogState:
    content= state['content']

    prompt= f"On the scale of 1 to 5 rate below blog.\n {content}. Also discuss on what basis you rate the blog."

    answer= model.invoke(prompt).content

    state['evaluate']= answer

    return state

In [14]:
# Graph Creation
graph= StateGraph(BlogState)

# Add Nodes
graph.add_node('outline_creation',outline_creation)
graph.add_node('content_creation', content_creation)
graph.add_node('evaluation', evaluate_response)

# Add Edges
graph.add_edge(START, 'outline_creation')
graph.add_edge('outline_creation','content_creation')
graph.add_edge('content_creation','evaluation')
graph.add_edge('evaluation', END)

# Graph Compile
workflow= graph.compile()

In [15]:
print(workflow.get_graph().draw_ascii())

    +-----------+    
    | __start__ |    
    +-----------+    
          *          
          *          
          *          
+------------------+ 
| outline_creation | 
+------------------+ 
          *          
          *          
          *          
+------------------+ 
| content_creation | 
+------------------+ 
          *          
          *          
          *          
   +------------+    
   | evaluation |    
   +------------+    
          *          
          *          
          *          
    +---------+      
    | __end__ |      
    +---------+      


In [16]:
initial_state= {"title":"what is important to human rather than family?"}

In [17]:
final_state= workflow.invoke(initial_state)

111111111111 Sorry, I don't have the capability to browse the internet for real-time information. Therefore, I am not able to provide specific details or sources on what is important to humans other than family. If you have any other questions or need assistance with general topics, feel free to ask.


In [19]:
print(final_state['evaluate'])

I would rate this blog a 4 out of 5. The content is well-written and addresses an important topic that many people can relate to. The blog covers various aspects beyond family that are important to humans, such as purpose, personal growth, social connections, and health. The points made are insightful and provide a comprehensive view of what contributes to a fulfilling life.

I rate this blog based on its relevance, clarity, and the depth of the content provided. The blog effectively discusses important aspects of human life beyond family, and offers valuable insights on how to lead a fulfilling and meaningful life. The writing is engaging and easy to follow, making it an enjoyable read. Overall, I believe this blog provides valuable information and perspectives on an important topic.


In [20]:
print(final_state['content'])

What is important to humans besides family? This is a question that has puzzled philosophers, psychologists, and everyday individuals alike. While family undoubtedly plays a vital role in our lives, there are other aspects that are equally important to human beings.

One of the most crucial things to humans is their sense of purpose and fulfillment. We all yearn to lead meaningful lives and make a positive impact on the world. Whether it be through our careers, passions, or relationships, finding purpose and satisfaction in what we do is essential to our overall well-being.

Furthermore, personal growth and self-improvement are also significant to humans. We are constantly seeking opportunities to learn, develop new skills, and evolve as individuals. Whether it be through education, travel, or personal experiences, the drive for self-improvement is innate within us.

Another important aspect to humans is social connections and relationships outside of the family unit. Friendships, roma